In [11]:
"""
Train on images split into directories. This assumes we've split
our videos into frames and moved them to their respective folders.
Use keras 2+ and tensorflow 1+
Based on:
https://keras.io/preprocessing/image/
and
https://keras.io/applications/
"""
import os
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K
from keras.optimizers import SGD, Adagrad, Adam, Nadam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Reshape, LSTM, TimeDistributed, Dropout, Input, Flatten, Lambda, PReLU
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
#from utils.clr.clr_callback import * 
#from UCFdata import DataSet
import math
import numpy as np

from keras import regularizers

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.models import Model
import os

In [2]:
from keras.layers import Dense, Wrapper 
import keras.backend as K 


class DropConnect(Wrapper): 
    def __init__(self, layer, prob=1., **kwargs): 
        self.prob = prob 
        self.layer = layer 
        super(DropConnect, self).__init__(layer, **kwargs) 
        if 0. < self.prob < 1.: 
            self.uses_learning_phase = True 

 
    def build(self, input_shape): 
        if not self.layer.built: 
            self.layer.build(input_shape) 
            self.layer.built = True 
        super(DropConnect, self).build() 

 
    def compute_output_shape(self, input_shape): 
        return self.layer.compute_output_shape(input_shape) 

 
    def call(self, x): 
        if 0. < self.prob < 1.: 
            self.layer.kernel = K.in_train_phase(K.dropout(self.layer.kernel, self.prob), self.layer.kernel) 
            self.layer.bias = K.in_train_phase(K.dropout(self.layer.bias, self.prob), self.layer.bias) 
        return self.layer.call(x) 


In [3]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


In [4]:
# Helper: Save the min val_loss model in each epoch.
checkpointer = ModelCheckpoint(
    filepath='incept(global)_lstm_pred.{epoch:03d}-{val_loss:.2f}-{val_acc:.2f}-1134-04172019.hdf5',
    verbose=1,
    monitor='val_acc',
    save_best_only=True)

# Helper: Stop when we stop learning.
# patience: number of epochs with no improvement after which training will be stopped.
#early_stopper = EarlyStopping(patience=10)

# Helper: TensorBoard
tensorboard = TensorBoard(log_dir='utils/logs/')

In [5]:
def get_generators():
    datagen = ImageDataGenerator(rescale=1. / 255)
    train_generator = datagen.flow_from_directory(
          'data/train',
          target_size=(224, 224),
          batch_size=batch_size,
          class_mode='categorical',  # this means our generator will only yield batches of data, no labels
          shuffle=False,
          classes=['angry','happy','sad','submissive'])

    validation_generator = datagen.flow_from_directory(
          'data/val',
          target_size=(224, 224),
          batch_size=batch_size,
          class_mode='categorical',  # this means our generator will only yield batches of data, no labels
          shuffle=False,
          classes=['angry','happy','sad','submissive'])
    return train_generator, validation_generator


In [17]:
def get_model(weights='imagenet'):
    # create the base pre-trained model
    base_model = InceptionV3(weights=weights, include_top=False)

    x2 = base_model.output
    x2 = DropConnect(Dense(2048, activation='relu'), prob=0.5)(x2)
    x2 = Reshape((25,2048))(x2)
    x2 = LSTM(256,dropout=0.2,input_shape=(25,2048))(x2)
    x2 = DropConnect(Dense(64, activation='relu'), prob=0.9)(x2)
    predictions = Dense(4, activation='softmax')(x2)
    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)    

    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer=Adagrad(lr = 0.0001, decay=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [7]:
def fine_tune_inception_layer(model):
    """After we fine-tune the dense layers, train deeper."""
    # we chose to train the top 2 inception blocks, i.e. we will freeze
    # the first 172 layers and unfreeze the rest:
    for layer in model.layers[:172]:
        layer.trainable = False
    for layer in model.layers[172:]:
        layer.trainable = True

    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    model.compile(
        optimizer=SGD(lr=0.0001, momentum=0.9),
        loss='categorical_crossentropy',
        metrics=['accuracy'])

    return model

In [8]:
def train_model(model, nb_epoch, generators, callbacks=[]):
    train_generator, validation_generator = generators
    model.fit_generator(
        train_generator,
        steps_per_epoch=23750/25,
        validation_data=validation_generator,
        validation_steps=7100/25,
        epochs=nb_epoch,
        callbacks=callbacks)
    return model


In [9]:
def main(weights_file):

    model = get_model()
    generators = get_generators()

#     if weights_file is None:
#         print("Training Top layers.")
#         model = train_model(model, 10, generators)
#     else:
#         print("Loading saved model: %s." % weights_file)
#         model.load_weights(weights_file)

#     # Get and train the mid layers.
#     print("Freezing Top Layers and Getting Mid Layers")
#     model = fine_tune_inception_layer(model)
    
    print("Training Mid layers")
    model = train_model(model, 10, generators,
                        [checkpointer, tensorboard])

In [18]:
batch_size = 25
if __name__ == '__main__':
    weights_file = None
    main(weights_file)
    

Found 23750 images belonging to 4 classes.
Found 7100 images belonging to 4 classes.
Training Mid layers
Epoch 1/10
950/950 [==============================] - 362s 381ms/step - loss: 1.4391 - acc: 0.4228 - val_loss: 1.4652 - val_acc: 0.3527

Epoch 00001: val_acc improved from -inf to 0.35268, saving model to incept(global)_lstm_pred.001-1.47-0.35-1134-04172019.hdf5
Epoch 2/10
950/950 [==============================] - 361s 380ms/step - loss: 1.5763 - acc: 0.2792 - val_loss: 1.4230 - val_acc: 0.3387

Epoch 00002: val_acc did not improve from 0.35268
Epoch 3/10
950/950 [==============================] - 366s 386ms/step - loss: 1.5256 - acc: 0.2777 - val_loss: 1.4163 - val_acc: 0.2806

Epoch 00003: val_acc did not improve from 0.35268
Epoch 4/10
950/950 [==============================] - 367s 386ms/step - loss: 1.5085 - acc: 0.2774 - val_loss: 1.3970 - val_acc: 0.3025

Epoch 00004: val_acc did not improve from 0.35268
Epoch 5/10
950/950 [==============================] - 366s 385ms/step -

# TESTING PURPOSE

In [19]:
"""
Classify test images set through our CNN.
Use keras 2+ and tensorflow 1+
It takes a long time for hours.
"""
import numpy as np
import operator
import random
import glob
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator


# CNN model evaluate
test_path = 'data/test' #path to your validation / test videos
test_data_gen = ImageDataGenerator(rescale=1. / 255)
batch_size = 25
test_generator = test_data_gen.flow_from_directory(test_path, target_size=(224, 224),
                                                   batch_size=batch_size, classes=['angry','happy','sad','submissive'],
                                                   class_mode='categorical',shuffle = False)


Found 14950 images belonging to 4 classes.


In [20]:
# load the trained model that has been saved in CNN_train_UCF101.py, your model name maybe is not the same as follow
model = load_model('incept(global)_lstm_pred.001-1.47-0.35-1134-04172019.hdf5', custom_objects={'DropConnect':DropConnect})

In [21]:
test_data_num = 14950 #the number of test images. use exactly number generated from generator.
test_generator.reset() #to avoid having bugs in generator.
# if you dont invoke .reset(), it will starts to mix the order of the array from predicted generator

predicted_array = model.predict_generator(generator=test_generator, steps=test_data_num/batch_size)

In [22]:
len(predicted_array) #check if generated data is the same as test_data_num

14950

# Manually calculate accuracy for model & template for documentation purposes

### Documentation section

In [23]:
frame = [] #check all frame names for each validation / test videos
filenames = [] #check all filenames for each validation / test videos
gt = [] #ground truth storage for each validation / test videos
length = [] #how many frames belongs to which test / validation videos
final_predicted_array = [] #final predicted results for all videos

catfiles = os.listdir(test_path)
for i in range(len(catfiles)):
    subvideos = os.listdir(os.path.join(test_path,catfiles[i]))
    filenames.extend(subvideos)
    for j in range(len(subvideos)):
        contentvideos = os.listdir(os.path.join(test_path,catfiles[i],subvideos[j]))
        frame.append(contentvideos)
        length.append(len(contentvideos))
        gt.append(i)
cum_length = np.cumsum(length)
test2 = predicted_array[:cum_length[0]]
final_predicted_array.append(test2)
for i in range(len(cum_length)-1):
    test2 = predicted_array[cum_length[i]:cum_length[i+1]]
    final_predicted_array.append(test2)
votepred = []
meanpred = []

for i in range(len(final_predicted_array)):
    votectg = np.bincount(np.argmax(final_predicted_array[i], axis=1))
    votepred.append(np.argmax(votectg))
    meanctg = np.argmax(np.mean(final_predicted_array[i], axis=0))
    meanpred.append(meanctg)

vote = 0
mean = 0
for i in range(len(gt)):
    if(gt[i] == votepred[i]):
        vote += 1
    if(gt[i] == meanpred[i]):
        mean += 1
        
vote_acc = vote/len(gt)
mean_acc = mean/len(gt)

print("voting accuracy : ", vote_acc)
print("mean accuracy : ", mean_acc)

voting accuracy :  0.27424749163879597
mean accuracy :  0.27424749163879597


# Looping method

Auto looping all available test / validation datasets

In [ ]:
for x in range(len(frame)):
    #print('filename:',frame[x])
    print(color.BOLD + filenames[x] + color.END)
    print('number of file:',length[x])
    #print('array:',final_predicted_array[x])
    avgstr = []
    for j in range(final_predicted_array[x].shape[1]):
        avg = np.mean(final_predicted_array[x][:,j])
        avgstr.append(avg) 
    predicted_labels, value = max(enumerate(avgstr), key=operator.itemgetter(1))
    print(avgstr)
    print('ground truth:',gt[x])
    print('prediction:', predicted_labels)
    print()

# For Video Voting

In [24]:
from PIL import Image
from random import randint
import random
import os


dirpath = 'data/test'
classes = os.listdir(dirpath)
length = []
ground_truth = []
ctr = 0
for i in range(len(classes)):
    classpath = os.path.join(dirpath,classes[i])
    clip = os.listdir(classpath)
    for clp in clip:
        clippath = os.path.join(classpath,clp)
        vids = os.listdir(clippath)
        test = int(clp[-2:]) * len(vids)
        if test < ctr:
            ground_truth.append(i)
            length.append(ctr)
            ctr = 0
        ctr = test
        
arraystorage = [[] for i in range(len(length))]
votepred = []
meanpred = []
for i in range(len(length)):
    if i == 0:
        arraystorage[i] = predicted_array[:length[i]]
    else:
        arraystorage[i] = predicted_array[length[i-1]:length[i]+length[i-1]]
        
        
for i in arraystorage:
    votectg = np.bincount(np.argmax(i, axis=1))
    votepred.append(np.argmax(votectg))
    meanctg = np.argmax(np.mean(i, axis=0))
    meanpred.append(meanctg)

vote = 0
mean = 0
for i in range(len(ground_truth)):
    if(ground_truth[i] == votepred[i]):
        vote += 1
    if(ground_truth[i] == meanpred[i]):
        mean += 1
        
vote_acc = vote/len(ground_truth)
mean_acc = mean/len(ground_truth)

print("voting accuracy : ", vote_acc)
print("mean accuracy : ", mean_acc)

voting accuracy :  0.25
mean accuracy :  0.25
